In [ ]:
import os
import time
import random
import numpy as np

import torch
import torch.nn as nn

from torch.autograd import Variable
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(CharRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        # Layers
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        encoded = self.encoder(x)
        y, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        y = self.decoder(y.view(batch_size, -1))
        return y, hidden

    def forward2(self, x, hidden):
        encoded = self.encoder(x.view(1, -1))
        y, hidden = self.rnn(encoded.view(1, 1, -1), hidden)
        y = self.decoder(y.view(1, -1))
        return y, hidden
    
    def init_hidden(self, batch_size):
        return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))

In [ ]:
vocab_size = 825
hidden_size = 32
chunk_size = 10
batch_size = 32
epochs = 100
learning_rate = 0.01

In [ ]:
rnn = CharRNN(vocab_size, hidden_size, vocab_size, n_layers=1)

In [ ]:
opt = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
loss = nn.CrossEntropyLoss()

In [ ]:
def get_batch(file, chunk_len, batch_size):
    
    file_len = len(file)
    
    input_ = torch.LongTensor(batch_size, chunk_len)
    target_ = torch.LongTensor(batch_size, chunk_len)
    
    for i in range(batch_size):
        
        start = np.random.randint(file_len - chunk_len)
        end = start + chunk_len + 1
        
        chunk = file[start:end]
        
        input_[i] = chunk[:-1]
        target_[i] = chunk[1:]
        
    input_ = Variable(input_)
    target_ = Variable(target_)
    
    return input_, target_

In [ ]:
def train(data_dir, epochs, batch_size, chunk_size):
    
    files = [os.path.join(data_dir, file) for file in 
             os.listdir(data_dir) if file.endswith('.npy')]
    
    pbar = tqdm(range(epochs), unit='epoch')
    
    for epoch in pbar:
        for file in files:
            
            f = torch.Tensor(np.load(file))
            file_len = len(f)
            
            pbar2 = tqdm(range(file_len), unit='example')
            
            for i in pbar2:
                
                hidden = rnn.init_hidden(batch_size)
                
                error = 0
                rnn.zero_grad()
                
                input_, target_ = get_batch(f, chunk_size, batch_size)
                
                for c in range(chunk_size):

                    output, hidden = rnn(input_[:,c], hidden)
                    error += loss(output.view(batch_size, -1), target_[:,c])
                    
                error.backward()
                opt.step()

In [ ]:
train('./data', epochs, batch_size, chunk_size)